In [1]:
from utils import get_urls_of_type
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import tqdm
import json
import os

In [2]:
type_dict = {
             'thoi-su': ['chinh-tri', 'dan-sinh', 'lao-dong-viec-lam', 'giao-thong', 'mekong', 'quy-hy-vong'],
             'goc-nhin': ['binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-suc-khoe', 'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong', 'van-hoa-doi-song', 'covid-19', 'tac-gia'],
             'the-gioi': ['bau-cu-tong-thong-my', 'tu-lieu', 'phan-tich', 'nguoi-viet-5-chau', 'cuoc-song-do-day', 'quan-su'],
             'kinh-doanh': ['net-zero', 'quoc-te', 'doanh-nghiep', 'chung-khoan', 'ebank', 'vi-mo', 'tien-cua-toi', 'hang-hoa'],
             'bat-dong-san': ['chinh-sach', 'thi-truong', 'khong-gian-song', 'tu-van'],
             'khoa-hoc': ['khoa-hoc-trong-nuoc', 'pii-doi-moi-sang-tao', 'tin-tuc', 'phat-minh', 'ung-dung', 'the-gioi-tu-nhien', 'thuong-thuc'],
             'giai-tri': ['gioi-sao', 'sach', 'video', 'phim', 'nhac', 'thoi-trang', 'lam-dep', 'san-khau-my-thuat'],
             'the-thao': ['bong-da', 'du-lieu-bong-da', 'marathon', 'tennis', 'cac-mon-khac', 'hau-truong'],
             'phap-luat': ['ho-so-pha-an', 'tu-van'],
             'giao-duc': ['tin-tuc', 'tuyen-sinh', 'chan-dung', 'du-hoc', 'thao-luan', 'hoc-tieng-anh', 'giao-duc-40'],
             'suc-khoe': ['tin-tuc', 'cac-benh', 'song-khoe', 'vaccine'],
             'doi-song': ['nhip-song', 'to-am', 'bai-hoc-song', 'cooking', 'tieu-dung'],
             'du-lich': ['diem-den', 'am-thuc', 'dau-chan', 'tu-van', 'cam-nang'],
             'so-hoa': ['cong-nghe', 'san-pham', 'blockchain'],
             'xe': ['thi-truong', 'dien-dan'],
             'y-kien': ['thoi-su', 'doi-song']
            }

In [3]:
os.makedirs("data/urls", exist_ok=True)

DESCRIPTION

In [ ]:
crawl_url_list = []
f_urls = open("data/urls/testurl_list.jsonl", "w", encoding="utf-8")

total_count = 0 
no_a_tag_in_a_row = 0
max_no_a_tag_in_a_row = 3

for type in tqdm.tqdm(type_dict, desc="Processing types"):
    for sub_type in tqdm.tqdm(type_dict[type], desc=f"Processing subtypes for {type}"):
        target_count = 5
        i = 1
        current_count = 0
        no_articles_in_a_row = 0
        
        while current_count < target_count:
            url = f"https://vnexpress.net/{type}/{sub_type}-p{i}"
            content = requests.get(url)

            if content.status_code != 200:
                print(f"{content.status_code} ERROR {type}|{sub_type}")
                break
            
            soup = BeautifulSoup(content.content, "html.parser")
            tmp_title_list = soup.find_all(class_="title-news")
            
            if len(tmp_title_list) == 0:
                no_articles_in_a_row += 1
                print(f"No article {type}|{sub_type}.")
                break
            else:
                no_articles_in_a_row = 0

                for title in tmp_title_list:
                    try:
                        article_info = title.find_all("a")
                        date_time = soup.find_all("span", class_="date")
                        
                        if article_info:
                            article_url = article_info[0].get("href")
                            article_title = article_info[0].get("title")
                            description = title.find_next("p", class_="description") 
                            description_text = "null"

                            if description:
                                description_text = description.get_text(strip=True)

                            sample = {
                                "url": article_url,
                                "title": article_title,
                                "type": f"{type}|{sub_type}",
                                "description": description_text
                            }
                            f_urls.write(json.dumps(sample, indent=4, ensure_ascii=False) + "\n")
                            current_count += 1
                            total_count += 1

                            if current_count % 5 == 0:
                                print(f"Total: {total_count}")

                            if current_count >= target_count:
                                break
                            
                            no_a_tag_in_a_row = 0
                            
                        else:
                            no_a_tag_in_a_row += 1
                            print(f"No <a> tag for {type}|{sub_type}. Continue...")
                            if no_a_tag_in_a_row >= max_no_a_tag_in_a_row:
                                print(f"No <a> tag for {type}|{sub_type}. Exiting...")
                                break
                            
                    except Exception as e:
                        print(f"ERROR: {e}")
                        pass
            
            if no_a_tag_in_a_row >= max_no_a_tag_in_a_row:
                print(f"No <a> tag for {type}|{sub_type}. Exiting....")
                break

            i += 1

        if no_a_tag_in_a_row >= max_no_a_tag_in_a_row:
            print(f"No <a> tag for {type}|{sub_type}. Exiting.....")
            break

f_urls.close()

Processing types:   0%|          | 0/16 [00:00<?, ?it/s]

Total: 5


Total: 10


Total: 15


Total: 20


...

Processing types:  94%|█████████▍| 15/16 [00:34<00:01,  1.60s/it]

404 ERROR xe|dien-dan


Total: 380


Processing types: 100%|██████████| 16/16 [00:35<00:00,  2.19s/it]

Total: 385
